## Inversion Recovery Example

In this example, we will perform an inversion-recovery experiment on a Pyridine molecule. This serves as an excellent demonstration of the functionality provided by the Spinguin package.

- **Molecule and Spins**: The simulation incorporates all NMR-active spins of the Pyridine molecule, including:
    - All 5 protons.
    - The quadrupolar nitrogen-14 nucleus.

- **Performance Optimization**: The computational performance is enhanced by utilizing the restricted Liouville space.

- **Interactions Included**:
    - **Coherent Interactions**:
        - Zeeman interaction.
        - Chemical shift.
        - Scalar $J$-coupling.
    - **Incoherent Interactions**:
        - Dipolar coupling.
        - Chemical shift anisotropy.
        - Quadrupolar coupling.
        - Scalar relaxation of the second kind.

This example provides a comprehensive overview of how to simulate NMR experiments using Spinguin.

### Imports

To begin, we import all the necessary modules.

- **NumPy**: Used for creating arrays required for input parameters and storing results.
- **Matplotlib**: Utilized for plotting the simulation results.
- **Spinguin**: Provides specialized functionality for simulating NMR experiments.

In [ ]:
# Import necessary modules:
import numpy as np
import matplotlib.pyplot as plt
from spinguin import SpinSystem, hamiltonian, relaxation, liouvillian, equilibrium_state, measure, propagator, pulse, transform_to_truncated_basis, truncate_basis_by_coherence

### Simulation Parameters

The following simulation parameters are defined:

- **Magnetic Field (T)**: The strength of the static magnetic field $B_0$.
- **Rotational Correlation Time (s)**: The characteristic time for molecular tumbling, affecting relaxation processes.
- **Temperature (K)**: The temperature at which the simulation is conducted.
- **Time Step (s)**: The interval between successive time points in the simulation.
- **Number of Time Steps**: The total number of time steps for the simulation.

In [ ]:
# Define simulation parameters:
B0 = 1  # 1 Tesla
tau_c = 10e-12  # 10 picoseconds
T = 295  # Kelvin
dt = 2e-3  # 2 milliseconds
N_steps = 50000

### Define the Spin System

To define the spin system, we need to specify the following parameters:

- **Isotopes**: The types of nuclei in the system.
- **Chemical Shifts (ppm)**: The chemical shifts for each nucleus.
- **J Couplings (Hz)**: The $J$-coupling constants between nuclei.
- **XYZ Coordinates (Å)**: The spatial positions of the nuclei in Cartesian coordinates.
- **Shielding Tensors (ppm)**: The nuclear shielding tensors for each nucleus.
- **EFG Tensors (a.u.)**: The electric field gradient tensors for quadrupolar nuclei.

These parameters are defined as NumPy arrays and provided as input when creating the spin system object. The maximum spin order is limited to 3 to enhance computational performance.

In [ ]:
# Define the spin system parameters:
isotopes = np.array(['1H', '1H', '1H', '1H', '1H', '14N'])
chemical_shifts = np.array([8.56, 8.56, 7.47, 7.47, 7.88, 95.94])

J_couplings = np.array([
    [ 0,     0,      0,      0,      0,      0],
    [-1.04,  0,      0,      0,      0,      0],
    [ 4.85,  1.05,   0,      0,      0,      0],
    [ 1.05,  4.85,   0.71,   0,      0,      0],
    [ 1.24,  1.24,   7.55,   7.55,   0,      0],
    [ 8.16,  8.16,   0.87,   0.87,  -0.19,   0]
])

xyz = np.array([
    [ 2.0495335,  0.0000000, -1.4916842],
    [-2.0495335,  0.0000000, -1.4916842],
    [ 2.1458878,  0.0000000,  0.9846086],
    [-2.1458878,  0.0000000,  0.9846086],
    [ 0.0000000,  0.0000000,  2.2681296],
    [ 0.0000000,  0.0000000, -1.5987077]
])

shielding = np.zeros((6, 3, 3))
shielding[5] = np.array([
    [-406.20,  0.00,    0.00],
    [   0.00, 299.44,   0.00],
    [   0.00,   0.00, -181.07]
])

efg = np.zeros((6, 3, 3))
efg[5] = np.array([
    [ 0.3069,  0.0000,  0.0000],
    [ 0.0000,  0.7969,  0.0000],
    [ 0.0000,  0.0000, -1.1037]
])

# Create the spin system object with the defined parameters.
# The maximum spin order is limited to 3 to enhance computational performance.
spin_system = SpinSystem(
    isotopes, 
    chemical_shifts, 
    J_couplings, 
    xyz, 
    shielding, 
    efg, 
    max_spin_order=3
)

### Calculate Hamiltonian, Relaxation Superoperator, and the total Liouvillian

To perform the time evolution of the spin system under the NMR interactions described by the Liouvillian, we need to calculate the following:

- **Hamiltonian**: Represents the coherent interactions in the system.
- **Relaxation Superoperator**: Accounts for the incoherent interactions and the resulting relaxation processes.

After calculating these, the total Liouvillian is assembled by combining the Hamiltonian and relaxation superoperator.

Other considerations:
- We include the scalar relaxation of the second kind for completeness, even though its effect at a magnetic field strength of 1 T is negligible.
- The relaxation superoperator is thermalized automatically when temperature is given as an input. This ensures that the spin system is driven back towards thermal equilibrium.

In [ ]:
# Calculate the Hamiltonian for the spin system.
H = hamiltonian(spin_system, B0)

# Calculate the relaxation superoperator for the spin system.
R = relaxation(spin_system, H, B0, tau_c, T, include_sr2k=True)

# Calculate the Liouvillian
L = liouvillian(H, R)

### Thermal Equilibrium

To perform an inversion recovery experiment, we need the thermal equilibrium state.

- The thermal equilibrium state represents the natural distribution of spin populations in the presence of an external magnetic field at a given temperature.
- We create a state vector corresponding to this thermal equilibrium, which serves as the starting point for the simulation.

In [ ]:
# Initialize the state vector to thermal equilibrium.
# TODO: This double Hamiltonian construction is not necessary.
rho = equilibrium_state(spin_system, T, B0)

### Apply 180-Degree Pulse

The inversion-recovery experiment begins with a 180-degree pulse.

- The 180-degree pulse is applied along the $x$-axis to each proton (spins with indices 0 to 4).
- This pulse is essentially a rotation operator that inverts the magnetization of the selected spins.
- The pulse is applied by multiplying the state vector with the rotation operator.

In [ ]:
# Apply a 180-degree pulse along the x-axis to the protons (spins with indices 0 to 4).
pul_180 = pulse(spin_system, "I(x,0) + I(x,1) + I(x,2) + I(x,3) + I(x,4)", angle=180)

# Update the state vector by applying the 180-degree pulse.
rho = pul_180 @ rho

In [ ]:
# TODO: I do not get the last point below.

### Change to Zero-Quantum Basis (Optional)

This step is optional but can significantly reduce computational cost by decreasing the dimensions of the matrices

- **State After Initial Pulse**: After the initial pulse, the state contains only zero-quantum terms. 
- **Evolution Under Liouvillian**: The evolution under the Liouvillian does not cause transitions that result in different coherence orders. 
- **Accuracy**: Restricting the basis to zero-quantum terms does not compromise accuracy in this case.
- **Order of Operations**: The conversion to the zero-quantum basis must be performed after the initial pulse. This is because the pulse along the $x$-axis is a mixture of zero- and double-quantum coherences.

Steps:
1. Apply the zero-quantum filter to the spin system.
2. Convert the Liouvillian and the state vector into the zero-quantum basis.

In [ ]:
# Generate the zero-quantum (ZQ) basis map for the spin system.
# This map identifies the zero-quantum terms in the system.
ZQ_map = truncate_basis_by_coherence(spin_system, [0])

# Filter the Hamiltonian, relaxation superoperator and state vector to retain only the zero-quantum terms.
L, rho = transform_to_truncated_basis(ZQ_map, L, rho)

### Construct the Time Propagator

The time propagator is a crucial component for simulating the time evolution of the spin system.

- **Purpose**: The propagator is used to drive the spin system forward in time during the simulation.
- **Construction**: It is constructed using the Liouvillian.
- **Time Step**: The propagator is specific to the chosen time step, ensuring accurate simulation of the dynamics.

In [ ]:
# Construct the time propagator for the simulation.
P = propagator(L, dt)

### Initialize an Empty Array for Storing Results

In an inversion-recovery experiment, we aim to measure the magnetizations of the spins at various time intervals.

- **Experimental Approach**: In real experiments, this is achieved by applying a 90-degree pulse and recording the free induction decay (FID).
- **Simulation Advantage**: In simulations, this step is unnecessary. We can directly observe the magnetization by "measuring" the expectation value of the $z$-magnetization of each spin at each time interval.
- **Storage**: To store the magnetizations, we initialize a NumPy array. The array dimensions correspond to:
    - The number of time steps in the simulation.
    - The number of protons (spins) in the spin system.

In [ ]:
# Initialize an array to store the magnetizations of the spins.
magnetizations = np.zeros((N_steps, 5), dtype=complex)

### Perform the Time Evolution

We are now ready to perform the time evolution of the spin system.

- **Loop Over Time Steps**: Iterate through the total number of time steps defined for the simulation.
- **Propagate the System**: At each time step, propagate the spin system forward in time using the time propagator.
- **Measure Magnetization**: For each proton (spins with indices 0 to 4), measure the $z$-magnetization and store the results in the `magnetizations` array.

In [ ]:
# Perform the time evolution of the spin system.
for step in range(N_steps):
    # Propagate the state vector to the next time step.
    rho = P @ rho

    # Measure the z-magnetization for each proton and store it.
    for idx in [0, 1, 2, 3, 4]:
        magnetizations[step, idx] = measure(spin_system, rho, f"I(z,{idx})")

### Plot the Results

Congratulations! The simulation is complete. The results are stored in the `magnetizations` array. You can now:

- Save the results as a CSV file for further analysis.
- Perform additional analysis directly on the results.
- Plot and visualize the results.

In this example, we will plot and display the results.

In [ ]:
# Generate a time array for the x-axis of the plot.
t = np.linspace(dt, N_steps * dt, N_steps)

# Plot the magnetization of each proton (spins with indices 0 to 4) over time.
for i in [0, 1, 2, 3, 4]:
    plt.plot(t, np.real(magnetizations[:, i]), label=f"Spin {i+1}")

# Add a legend to identify each spin's magnetization curve.
plt.legend(loc="upper right")

# Label the x-axis and y-axis for clarity.
plt.xlabel("Time (s)")
plt.ylabel("Magnetization")

# Add a title to describe the plot.
plt.title("Inversion-Recovery of Pyridine")

# Adjust the layout to prevent overlapping of labels and elements.
plt.tight_layout()

# Display the plot.
plt.show()

# Clear the figure to avoid overlapping plots in subsequent cells.
plt.clf()